<a href="https://colab.research.google.com/github/steven4354/jsonformer-test/blob/main/jsonformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install jsonformer

from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set the folder path in Google Drive for model files
google_drive_model_path = "/content/drive/My Drive/Colab Notebooks/dolly-v2-12b"

# Check if the model folder exists
if not os.path.exists(google_drive_model_path):
    # Create folder in Google Drive
    os.makedirs(google_drive_model_path)
    
    # Download model and tokenizer
    model_name = "databricks/dolly-v2-12b"
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Save model and tokenizer to Google Drive
    model.save_pretrained(google_drive_model_path)
    tokenizer.save_pretrained(google_drive_model_path)

# Load model and tokenizer from Google Drive
model = AutoModelForCausalLM.from_pretrained(google_drive_model_path)
tokenizer = AutoTokenizer.from_pretrained(google_drive_model_path)

# Use the loaded model and tokenizer with Jsonformer
from jsonformer import Jsonformer

json_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "number"},
        "is_student": {"type": "boolean"},
        "courses": {
            "type": "array",
            "items": {"type": "string"}
        }
    }
}

prompt = "Generate a person's information based on the following schema:"
jsonformer = Jsonformer(model, tokenizer, json_schema, prompt)
generated_data = jsonformer()

print(generated_data)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
